In [90]:
import PyPDF2
import re
import spacy
from spacy import displacy
import en_core_web_sm
nlp = en_core_web_sm.load()
import os
import nltk
import pandas as pd
import docx2txt
import datetime

In [49]:
resumes = os.listdir('./resumes')
print ('Number of resumes found: {}'.format(len(resumes)))

Number of resumes found: 13


In [50]:
applicant_path = list()
applicant_name = list()
applicant_email = list()
applicant_phone = list()

counter = 0
for resume in resumes:
    counter = counter + 1
    if resume.split('.')[1] =='pdf':
        text_all = ''
        pdf_file = open('./resumes/' + resume, 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdf_file)
        count = pdfReader.numPages
        for i in range(count):
            page = pdfReader.getPage(i)
            text_all = text_all + ' ' + page.extractText()
    else:
        text_all = docx2txt.process('./resumes/' + resume)
    
    text = text_all.replace(':',': ')
    email = re.findall('\S+@\S+', text)  
    
    #print('Emaild found: {}'.format(email[0]))
    applicant_email.append(email[0])
    applicant_path.append('./resumes/' + resume)
    
    text = text_all.replace('-','')
    text = text.title()
    text = text.replace(':','')
    text = text.replace('\n','')
    text = text.replace('.','')
    text = text.replace('(','')
    text = text.replace(')','')
    text = text.replace('!','')
    phone = [x for x in re.findall('\d+',text) if len(x)>6]
    #print ('Contact Number found: {}'.format(phone[0]))
    applicant_phone.append(phone[0])
    
    
    extract_names = list()

    for sent in nltk.sent_tokenize(text):
        tokens = nltk.tokenize.word_tokenize(sent)
        tags = st.tag(tokens)
        if len(extract_names)>1:
            break
        else:
            for tag in tags:
                if tag[1]=='PERSON':
                    extract_names.append(tag[0])
                    if (len(extract_names))>1:
                        break   
    try:
        name = extract_names[0] + ' ' + extract_names[1]
    except:
        name = 'Not Found'
    #print('Name found: {}'.format(name))
    applicant_name.append(name)
    
    print('Completed {} out of {}'.format(counter, str(len(resumes))), end = '          \r')
    
df = pd.DataFrame()
df['applicant_path'] = applicant_path                           
df['applicant_name'] = applicant_name 
df['applicant_phone'] = applicant_phone
df['applicant_email'] = applicant_email
df.to_excel('resume_summary.xlsx')

In [115]:
def extract_resume_info(full_path_to_resume_folder, full_path_to_output_folder):
    
    resumes = os.listdir(full_path_to_resume_folder)
    print ('Number of resumes found: {}'.format(len(resumes)))
    
    applicant_path = list()
    applicant_name = list()
    applicant_email = list()
    applicant_phone = list()
    
    counter = 0
    for resume in resumes:
        counter = counter + 1
        if resume.split('.')[1] =='pdf':
            text_all = ''
            pdf_file = open(full_path_to_resume_folder + '/' + resume, 'rb')
            pdfReader = PyPDF2.PdfFileReader(pdf_file)
            count = pdfReader.numPages
            for i in range(count):
                page = pdfReader.getPage(i)
                text_all = text_all + ' ' + page.extractText()
        else:
            text_all = docx2txt.process(full_path_to_resume_folder + '/' + resume)
        
        text = text_all.replace(':',': ')
        email = re.findall('\S+@\S+', text)  
        
        #print('Emaild found: {}'.format(email[0]))
        applicant_email.append(email[0])
        applicant_path.append(full_path_to_resume_folder + '/' + resume)
        
        text = text_all.replace('-','')
        text = text.title()
        text = text.replace(':','')
        text = text.replace('\n','')
        text = text.replace('.','')
        text = text.replace('(','')
        text = text.replace(')','')
        text = text.replace('!','')
        phone = [x for x in re.findall('\d+',text) if len(x)>6]
        #print ('Contact Number found: {}'.format(phone[0]))
        applicant_phone.append(phone[0])
        
        
        extract_names = list()
    
        for sent in nltk.sent_tokenize(text):
            tokens = nltk.tokenize.word_tokenize(sent)
            tags = st.tag(tokens)
            if len(extract_names)>1:
                break
            else:
                for tag in tags:
                    if tag[1]=='PERSON':
                        extract_names.append(tag[0])
                        if (len(extract_names))>1:
                            break   
        try:
            name = extract_names[0] + ' ' + extract_names[1]
        except:
            name = 'Not Found'
        #print('Name found: {}'.format(name))
        applicant_name.append(name)
        
        print('Completed {} out of {}'.format(counter, str(len(resumes))), end = '          \r')
        
        
    df = pd.DataFrame()
    df['applicant_path'] = applicant_path                           
    df['applicant_name'] = applicant_name 
    df['applicant_phone'] = applicant_phone
    df['applicant_email'] = applicant_email
    
    extract_current_time = datetime.datetime.now()
    append_datetime = str(extract_current_time.year)+str(extract_current_time.month)+str(extract_current_time.day)+str(extract_current_time.hour)+str(extract_current_time.minute)+str(extract_current_time.second)
    
    df.to_excel(full_path_to_output_folder + '/' +'resume_summary_' + append_datetime +'.xlsx')

In [27]:
#doc = nlp(text)
    #names = [X.text for X in doc.ents if X.label_=='PERSON']
    #for i in range(len(names)):
    #    cond = (len(names[i].strip()) - len(names[i].strip().replace(' ','')))
    #    name = 'Not Found'
    #    if cond >0 and cond <4 and len(names[i])<30:
    #        name = names[i]
    #    break

'DARON ACEMOGLU'

In [119]:
extract_resume_info('D:\\AnuragHalder\\Learning\\Resume Extraction\\resumes', 'D:\\AnuragHalder\\Learning\\Resume Extraction')

Number of resumes found: 13
